In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import json

In [2]:
with open("source_texts/bible/luther_old_testament.xml", "r") as f:
    bible_ot = f.read()

In [31]:
soup = BeautifulSoup(bible_ot, 'xml')

In [155]:
def chunk_verses(verse_text: str) -> list:
    all_chunks = []
    chunks = re.split('/|,|\.|:', verse_text)
    word_collector = ""
    for chunk in chunks:
        word_collector += " " + chunk.strip()
        if len(word_collector.split()) >= 4:
            all_chunks.append(word_collector.strip())
            word_collector = ""
    return all_chunks

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_518/2323431692.py:3: SyntaxWarning: invalid escape sequence '\.'
  chunks = re.split('/|,|\.|:', verse_text)


In [147]:
old_testament = {}
books = soup.find_all("BIBLEBOOK")

for book in books:
    booktitle = book['bname']
    filename = f"source_texts/bible/old_testament/{re.sub(" ", "_", booktitle)}.json"
    chapters = {}
    for chap_nr in range(len(book.find_all("CHAPTER"))):
        chapter = book.find_all("CHAPTER")[chap_nr]
        verses = {}
        for verse_nr in range(len(chapter.find_all("VERS"))):
            verse = chapter.find_all("VERS")[verse_nr]
            for note in verse.find_all("NOTE"):                     # remove annotations
                note.string.replace_with(" ")
            verstext = verse.get_text()
            verse_chunked = chunk_verses(verstext)
            verses[verse_nr + 1] = verse_chunked
        chapters[chap_nr + 1] = verses
    old_testament[booktitle] = chapters

with open("source_texts/bible/old_testament_chunked.json", "w") as f:
    json.dump(old_testament, f,ensure_ascii=False)


In [181]:
with open("source_texts/bible/luther_septembertestament_1522.TEI-P5.xml", "r") as f:
    bible_tei = f.read()

In [182]:
rep = {"&#x017F;": "s",
       "&#xA75B;": "r",
       "vn&#x0303;": "vnd",
       "&#x0303;": "n",
       "<lb/>": " "
            }

rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
bible_tei = pattern.sub(lambda m: rep[re.escape(m.group(0))], bible_tei)

bible_tei = re.sub(r'<ref target="https:.*<\/ref>', " ◊ ", bible_tei)

soup = BeautifulSoup(bible_tei, 'xml')

In [183]:
bible_nt_books = ["Mt", "Mk", "Lk", "Joh",
                  "Apg", "Röm", "Kor1", "Kor2",
                  "Gal", "Eph", "Phil", "Kol",
                  "Thess1", "Thess2", "Tim1", "Tim2",
                  "Tit", "Phlm", "Petr1", "Petr2",
                  "Joh1", "Joh2", "Joh3", "Hebr",
                  "Jak", "Jud", "Offb"]

In [184]:
for note in soup.find_all("note"):      # remove all comments not editorial
    if note.get("type") != "editorial":
        note.decompose()

for fw in soup.find_all("fw"):
    fw.decompose()

for head in soup.find_all("head"):
    head.decompose()

for sic in soup.find_all("sic"):
    sic.decompose()

for reg in soup.find_all("reg"):
    reg.decompose()

In [185]:
books = soup.find_all("div", {"xml:id": lambda x: x in bible_nt_books})

In [186]:
new_testament = {}

url = re.compile(r'https?://\S+')

for book in books:
    booktitle = book["xml:id"]
    print(f"Book: {booktitle}")
    filename = f"source_texts/bible/new_testament/{booktitle}.json"
    chapters = {}
    for chap_nr in range(len(book.find_all("div"))):
        print(f"Chapter: {chap_nr+1}")
        chapter = book.find_all("div")[chap_nr]
        text = chapter.get_text(strip=True)
        text = re.sub("\n", "", text)
        text = re.sub(r"- ", "", text)
        text = re.sub("-", "", text)

        verse_nr = 0
        verses = {}
        
        split_text = text.split("◊")
        for elem in split_text:
            if elem != "":
                verse_nr += 1
                print(f"Verse: {verse_nr}, {elem}")
                verse_chunked = chunk_verses(elem)
                verses[verse_nr] = verse_chunked
        chapters[chap_nr + 1] = verses
        vers_nr = 0
    new_testament[booktitle] = chapters

        #sents = [f"{x.strip()}." for x in par_text.split(".")][:-1]
        #sents = [f"{x.strip()}." for x in par_text.split(".")]
        #par_list.append(sents)
        #chapter_list.append(par_list)


Book: Mt
Chapter: 1
Verse: 1, DJs ist das buch von der gepurt Jhesu Christi derdo ist ein son Dauids des sons  Abraham.
Verse: 2, Abraham hat geporn den Jsaac. Jsaac hatt geporn den Jacob. Jacob hatt geporn den Juda vnnd  seyne bruder.
Verse: 3, Juda hat geporn den Pharez vnnd  den Zaram von der Thamar. Pharez hat geporn den Hezron. Hezron hat geporn den Ram.
Verse: 4, Ram hat geporn den Amminadab. Amminadab hatt geporn den Nahasson. Nahasson hatt geporn den Salma.
Verse: 5, Salma hat geporn den Boas von der Rahab. Boas hat geporn den Obed von der Rhut. Obed hat geporn den Jesse.
Verse: 6, Jesse hatt geporn den konig Dauid.Der konig Dauid hat geporn den Salomon vonn dem weybe des Vrie.
Verse: 7, Salomon hat geporn den Roboam. Roboam hat geporn den Abia. Abia hat geporn den Assa.
Verse: 8, Assa hat geporn den Josaphat. Josaphat hat geporn den Joram. Joram hat geporn den Osia.
Verse: 9, Osia hat geporn den Jotham. Jotham hat geporn den Achas. Achas hat geporn den Ezechia.
Verse: 10, Ezec

In [187]:
new_testament

{'Mt': {1: {1: ['DJs ist das buch von der gepurt Jhesu Christi derdo ist ein son Dauids des sons  Abraham'],
   2: ['Abraham hat geporn den Jsaac',
    'Jsaac hatt geporn den Jacob',
    'Jacob hatt geporn den Juda vnnd  seyne bruder'],
   3: ['Juda hat geporn den Pharez vnnd  den Zaram von der Thamar',
    'Pharez hat geporn den Hezron',
    'Hezron hat geporn den Ram'],
   4: ['Ram hat geporn den Amminadab',
    'Amminadab hatt geporn den Nahasson',
    'Nahasson hatt geporn den Salma'],
   5: ['Salma hat geporn den Boas von der Rahab',
    'Boas hat geporn den Obed von der Rhut',
    'Obed hat geporn den Jesse'],
   6: ['Jesse hatt geporn den konig Dauid',
    'Der konig Dauid hat geporn den Salomon vonn dem weybe des Vrie'],
   7: ['Salomon hat geporn den Roboam',
    'Roboam hat geporn den Abia',
    'Abia hat geporn den Assa'],
   8: ['Assa hat geporn den Josaphat',
    'Josaphat hat geporn den Joram',
    'Joram hat geporn den Osia'],
   9: ['Osia hat geporn den Jotham',
    'Jo

In [188]:
with open("source_texts/bible/new_testament_chunked.json", "w") as f:
    json.dump(new_testament, f,ensure_ascii=False)

In [119]:
chapter_list

[[['DJs ist die offinbarungJhesu Christi.',
   'die yhmgott geben hatt/ seynen knechten zuzeygen/ was ynn der kurtz geschehensoll/ vnd hatt sie deuttet vnd gesanddurch seynen engel zu seynem knechtJohannes/ der betzeuget hatt daswort Gottis/ vnnd des zeugnis vonJhesu Christo/ was er gesehen hat/Selig ist der da lisset vñ die da horẽdie wort der weyssagung/ vnd behalbehalten was drynnen geschrieben ist/ deñdie zeyt ist nahe.'],
  ['Johannes/ Den sieben gemeynenynn Asia/ Gnad sey mit euch vñ fride/ võ dem der da ist/ vnd der dawar/ vnd der da kompt/ vnd von den sieben geystern/ die da sind furseynem stuel/ vnd von Jhesu Christo/ wilcher ist der trewe zeuge vnderst geporner von den todten/ vnd eyn Furst aller konige auff erden/der vns geliebt hat vnd gewaschen von den sunden mit seynem blut/vnd hat vns zu konigen vnd priestern gemacht fur Gott vnnd seynemvater/ dem selbigen sey preysz vnd reich von ewickeyt zu ewickeyt AmẽSihe/ er kompt mit den wolcken/ vnnd es werden yhn sehen alle